# DALI Lab: Data Challenge
By Youmi Ji (youmi.ji.28@dartmouth.edu)<br>
July 12, 2025<br>
Challenge prompt: Scientists with the National Park Service are researching barnacle populations in coastal tide pools on the east coast. To analyze the results of their experiments, they often need to count the number of barnacles in a given area. To do this, they place a fixed size frame on a barnacle-covered rock, then take a picture. Later, a scientist/lab tech will manually count the number of barnacles in the picture, then record their results. There are often upwards of 1000 barnacles in an image, so this is a very time consuming process. These scientists have now come to you to speed up their pipeline. **What system can you develop to help them process these images faster?**

## Step 1: Rough idea
**Files:**<br>
* img1.png, img2.png = sample images ("samp1" and "samp2")<br>
* mask1.png, mask2.png = manual annotations for sample images ("ant1" and "ant2")<br>
* masked_img1.png, masked_img2.png = samp + ant ("key1" and "key2")<br>
* unseen_img1.png = unlabeled sample ("test")<br>

**Brainstorm notes:** <br>
The main idea is to develop a system that automatically detects and counts barnacles in images taken in the field. Since we only have two manually labeled examples (img1 + mask1 and img2 + mask2), we plan to use these as our foundational data. By comparing the pixel locations in the sample images (samp1 and samp2) with their corresponding manual annotations (ant1 and ant2), we can train a model to learn what barnacle areas look like. Once trained, this system will be used to identify and count barnacles in new, unlabeled images (like test). <br>

# Step 2: Subtasks
Some critical subtasks necessary for developing this program include:<br>
* **Identify and isolate the green sampling frame**: Locates the green wire frame in each image; crops the image so we only analyze the area inside the frame, where barnacle counts matter.
* **Detect and segment barnacle areas**: Using a program such as Convolutional neural networks (CNN) or Random Forest for image classification and object recognition tasks. This subtask would be responsible for training as well as checking the accuracy (ie. through a confusion matrix and observing the f1-score)
* **Evaluate the system’s performance**: Compares the model’s predictions to the manual annotations; computes metrics like precision, recall, and F1-score to assess how accurately the system identifies barnacle regions.



# Step 3: Brainstorming subtask (model-focused approach)
For this part of the project, I wanted to focus specifically on automatically estimating the number of barnacles using feature detection and clustering, and then evaluating how close this estimate is to the true count from the annotations. Here’s the rough plan for the final version of this subtask, directly based on the approach implemented in my estimate and compare_to_key functions.

I start by loading the barnacle images in grayscale along with their corresponding masks. The masks are processed into a binary form to clearly isolate the regions containing barnacles. Then, I use the ORB feature detector to extract keypoints — local distinctive points that capture textural or structural information in the image. By filtering these keypoints to only include those that fall within the masked barnacle regions, I ensure that the subsequent analysis focuses on areas of interest.

Next, I apply DBSCAN clustering to group the spatial coordinates of these keypoints. Each cluster ideally corresponds to an individual barnacle, so the total number of clusters provides an estimate of the barnacle count. Based on earlier trials, I fixed the parameters to eps=5 and min_samples=3 since these produced the most consistent results.

To evaluate how accurate this estimated count is, I use the compare_to_key function. This function loads a color-coded image where barnacles have been manually annotated in blue, uses thresholding to isolate these annotations, then extracts their contours to count the actual number of barnacles present. By comparing the estimated count to this ground truth, I can compute a simple percent error, giving a quantitative sense of how well this method performs.

Pseudocode:
1. Load the barnacle image and its mask in grayscale.
2. Convert the mask into a binary mask that highlights barnacle regions.
3. Use ORB to detect keypoints in the image and compute their descriptors.
4. Filter the keypoints to retain only those that fall inside the binary mask regions.
5. Apply DBSCAN clustering (using fixed parameters eps=5, min_samples=3) to group the keypoints into clusters.
6. Count the number of clusters, which serves as the estimated number of barnacles.
7. Load the masked image with blue annotations and use color thresholding to extract these regions.
8. Find contours of the blue annotations to determine the actual number of barnacles.
9. Compute and print the percent error between the estimated count and the actual count to evaluate performance.

#Step 4: Prototype!
Over the course of this project, I explored several different approaches to analyze and classify the barnacle images, iteratively refining my methods based on the limitations and insights of each stage. I began by implementing a convolutional neural network (CNN), leveraging its capacity to automatically extract spatial hierarchies of features from the images. However, given the relatively small dataset and the irregular nature of the patterns, the CNN struggled to generalize effectively, resulting in underwhelming performance.

Seeking a more interpretable model that could potentially handle limited data better, I next experimented with a Random Forest classifier. For this approach, I focused on engineering features manually—hoping the ensemble of decision trees would be robust to noise and capture key distinguishing characteristics. Unfortunately, without well-defined or consistently strong features, this strategy also failed to deliver meaningful separation or reliable predictions.

Recognizing these challenges, I shifted to an unsupervised approach that did not rely on explicit labels. I employed ORB (Oriented FAST and Rotated BRIEF) to detect and describe keypoints in the images, allowing me to extract local patterns and structural signatures directly from the data. To identify underlying groupings, I then used DBSCAN, a density-based clustering algorithm well-suited for discovering arbitrary-shaped clusters and handling noise. This combination provided a more natural way to uncover similarities and differences among the barnacle samples, aligning better with the intrinsic structure of the data.

Through this progression—from supervised deep learning to classical ensemble methods, and finally to feature-based unsupervised clustering—I was able to adaptively tailor the methodology to the nature of the problem and dataset at hand.<br>

Assumptions:<br>
First, this program assumes that the input images (img1, img2) and their corresponding masks (masked_img1, masked_img2) are aligned in both scale and cropping. In this case, img1 and img2 were manually cropped to match the dimensions of their masked counterparts. (In a more robust pipeline, this alignment step could be automated as a separate preprocessing subtask to dynamically detect and crop the green wire frame area.) The program also assumes relatively consistent lighting and barnacle contrast, so that ORB can reliably detect keypoints, and that DBSCAN clustering will effectively separate individual barnacles based on these features. Finally, it presumes that the annotated blue regions in the masked images provide a close approximation of ground truth barnacle locations for evaluation.<br>


<br>References:
<br>https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html
<br>https://docs.opencv.org/4.x/d1/d89/tutorial_py_orb.html

In [ ]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN

def estimate(img_file, mask_file, nfeatures=13500):
    # Based on earlier experiments trying different nFeature, eps, and min_samples values, it seems that using
    # nFeature=13500, eps=5, and min_samples=3 produces the most accurate estimates.

    img = cv.imread(img_file, cv.IMREAD_GRAYSCALE)
    mask = cv.imread(mask_file, cv.IMREAD_GRAYSCALE)
    _, mask_bin = cv.threshold(mask, 127, 1, cv.THRESH_BINARY)

    orb = cv.ORB_create(nfeatures=nfeatures)
    keypoints = orb.detect(img, None)
    keypoints, descriptors = orb.compute(img, keypoints)

    points = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        if 0 <= y < mask_bin.shape[0] and 0 <= x < mask_bin.shape[1]:
            if mask_bin[y, x] == 1:
                points.append([x, y])
    points = np.array(points)

    if len(points) > 0:
        db = DBSCAN(eps=5, min_samples=3).fit(points)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        print(f"\nEstimate: {n_clusters} barnacles")
        return n_clusters
    else:
        print("\nError: No ORB keypoints inside barnacle mask. Try adjusting ORB or mask thresholding.")
        return 0

def compare_to_key(masked_img_file, estimated_count):
    masked_img = cv.imread(masked_img_file)
    masked_bin = cv.inRange(masked_img, (0,0,128), (100,100,255))
    contours, _ = cv.findContours(masked_bin, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    actual_barnacle_count = len(contours)
    print(f"Actual number of barnacles (from blue annotations): {actual_barnacle_count}")

    if actual_barnacle_count > 0:
        error_pct = abs(estimated_count - actual_barnacle_count) / actual_barnacle_count * 100
        print(f"\nEstimated: {estimated_count}, Actual: {actual_barnacle_count}")
        print(f"Percent error: {error_pct:.2f}%")
    else:
        print("\nError: No blue annotations detected in masked image — cannot compute percent error.")


In [ ]:
# testing "estimate" and "compare_to_key" on img1.png
print("\nTesting on img1:")
estimate1 = estimate("img1.png", "mask1.png")
compare_to_key("masked_img1.png", estimate1)

# testing "estimate" and "compare_to_key" on img2.png
print("\nTesting on img2: ")
estimate2 = estimate("img2.png", "mask2.png")
compare_to_key("masked_img2.png", estimate2)

# testing "estimate" on unseen_img1
print("\nTesting on unseen_img1:")
estimate_unseen1 = estimate("unseen_img1.png", "mask1.png")

#  testing "estimate" on unseen_img2.png
print("\nTesting on unseen_img2:")
estimate_unseen2 = estimate("unseen_img2.png", "mask2.png")



Testing on img1:

Estimate: 1361 barnacles
Actual number of barnacles (from blue annotations): 0

Error: No blue annotations detected in masked image — cannot compute percent error.

Testing on img2: 

Estimate: 1387 barnacles
Actual number of barnacles (from blue annotations): 1391

Estimated: 1387, Actual: 1391
Percent error: 0.29%

Testing on unseen_img1:

Estimate: 1708 barnacles

Testing on unseen_img2:

Estimate: 203 barnacles


# Step 5: Results
Both the *estimate* and *compare_to_key* functions had excellent results when testing img2. The *estimate* function was able to identify 1387 barnacles from img2.png, and after comparing to the actual number of barnacles that the scientists counted in masked_img2.png, the system had an error of 0.29%. Although my program wasn't able to count the actual number of barnacles using masked_img1, the number seems to be correct just from observing the density of barnacles with my eyes. Though there is not masked images (a.k.a. "answer key") for the two unseen images, it seems from the density/size of the image that these estimates are most likely accurate.

#Step 6: Conclusions
Overall, my prototype suggests that this approach — using ORB to extract local keypoints and DBSCAN to cluster them — is a promising way to analyze barnacle populations on a small scale. It handled the typical shapes and densities in the provided images quite well, and was able to differentiate and count barnacle-like regions effectively.

That said, there are still considerations that need to be explored before deciding if this is truly viable for broader use:

* **Scalability to larger areas:** It’s unclear how this pipeline would hold up if we applied it to much bigger sampling frames, where the number of barnacles (and therefore detected keypoints) could explode. DBSCAN’s behavior depends heavily on the density of points, so in more crowded or expansive images, we might need extensive parameter tuning — or even a shift to a hierarchical or adaptive clustering method.

* **Robustness to variation:** So far, this worked well because the barnacles were relatively uniform. But barnacles can vary a lot in shape and size depending on their environment. If we start seeing irregular or overlapping forms, ORB might fail to detect them consistently, or DBSCAN might accidentally merge distinct barnacles into one cluster.

Given all that, I think this prototype is a solid foundation and worth pursuing. But before turning it into a tool used in the pipeline, I’d recommend:

* Testing it on larger, more complex images with different barnacle densities to see how it scales.
* Adding shape or texture features alongside ORB keypoints to better capture variation in barnacle morphology.
* Looking into adaptive clustering techniques (like HDBSCAN) that can adjust to changing local densities automatically.

Longer term, it could also be exciting to build a visualization tool that lets scientists manually verify and correct the machine’s detections. Their corrections could feed back into the system, essentially teaching it what it missed. This kind of reward-based learning would help the program get better over time.


#Step 7: What I learned!
This problem was tough — honestly pretty frustrating at times — but that’s exactly what made it so rewarding! It pushed me well outside my comfort zone.  I went into it initially trying to apply methods I was familiar with from my CS 72 coursework: first testing a CNN (thinking of it like a visual pattern recognizer similar to how we approached language data) and then experimenting with a Random Forest on extracted pixel or contour features. However, both of these approaches were ill-suited to the specific visual clustering problem of barnacles, and my early attempts failed to generalize or even meaningfully detect individual barnacles.

What I loved, though, was how this forced me to rethink the problem entirely. Working through tutorials and exploring computer vision libraries felt like being handed a pile of building blocks — my job was to figure out how to get them to stand in just the right configuration. Shifting from standard ML models to ORB for keypoint detection and DBSCAN for clustering was a huge departure from my initial plan, but it made me realize how often real-world problems require stepping outside of familiar paradigms.

It also showed me the importance of iteration and parameter tuning. My first ORB + DBSCAN run produced very poor results, but by experimenting with values like nFeatures for ORB and eps/min_samples for DBSCAN, I could substantially improve the counts.

More broadly, I learned that working on projects like this is largely about finding shortcuts and creatively identifying more direct, effective solutions, even if they’re outside your comfort zone. This challenge taught me to be open-minded and ready to start over from scratch with an approach I didn’t initially expect to use. It was satisfying to see how general concepts I’ve learned in machine learning could translate into a completely different type of problem-solving.